(clas)=
# Clasificación de eventos
Para comparar los algoritmos de las LHCO 2020, utilizamos como base algunos algoritmos sencillos ya implementados en librerías como `scikit-learn`{cite}`scikit-learn` y uno programado usando `TensorFlow`{cite}`tensorflow2015-whitepaper`. En esta sección, se observarán algunas de las característica de la clasificación realizada con estos algoritmos. Los modelos utilizados son los presentados en la {numref}`clas-alg`, explicados en la {numref}`alg`.

```{table} Algoritmos utilizados para comparación
:name: clas-alg

|      Nombre                          | Implementación | Tipo                                             |
|:------------------------------------:|:--------------:|:------------------------------------------------:|
|Random Forest Classifier (RFC)        | `scikit-learn` | [Bosque aleatorio](alg-bosques)                  |
|Gradient Boosting Classifier (GBC)    | `scikit-learn` | [Clasificador del gradiente del impulso](alg-gbc)|
|Quadratic Discriminant Analysis (QDA) | `scikit-learn` | [Análisis de discriminante cuadrático](alg-qda)  | 
|MLP Classifier                        | `scikit-learn` | [Red neuronal](alg-neural)                       |
|Tensorflow Classifier                 | `tensorflow`   | [Red neuronal](alg-neural)                       |
| KMeans                               | `scikit-learn` | [K-means](alg-kmeans)                            |
```

Para el entrenamiento de los modelos y realizar las predicciones se utilizaron las variables descritas en la {numref}`bench-variables`, a excepción de las variables de masa ($m_{jj}$, $m\_{j_1}$ y $m\_{j_2}$), con el fin de intentar una clasificación libre de modelo en lo que respecta a la masa de las partículas.

(clas-RnD)=
## Conjunto R&D
Los datos del conjunto R&D se dividieron en conjuntos mutuamente excluyentes: 70% en un conjunto de entrenamiento y 30% en uno de prueba. A continuación se mostrará la importancia de las variables para la clasificación según RFC y GBC y se observarán las distribuciones de los eventos clasificados para algunas variables, utilizando el conjunto de prueba.

### Importancia de las características
De los modelos en la {numref}`clas-alg`, RFC y GBC permiten conocer cuáles de las variables de los eventos fueron las más relevantes para discriminar entre clases. Esto algoritmos asignan puntajes a las variables de acuerdo a su importancia para la clasificación. Un gráfico de estos puntajes se observa en la {numref}`clas-feature-imp`.

A continuación, realizaremos el entrenamiento de los modelos y la clasificación de los datos de prueba. Las celdas siguientes preparan los datos, entrenan los modelos y realizan la clasificación utilizando funciones de `benchtools`.

In [1]:
# Importamos las librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import argparse
import os.path

# De scikit-learn importamos herramientas
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
# Y los clasificadores
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans

# Lo necesario para construir el modelo de tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

# Funciones de benchtools
from benchtools.src.plotools import pred_test_hist, image_grid
from benchtools.src.clustering import build_features
from benchtools.src.datatools import separate_data
from benchtools.scripts.run import TensorflowClassifier, training, evaluate
from benchtools.src.metrictools import rejection_plot, precision_recall_plot

# Definimos semillas para la reproducibilidad
tf.random.set_seed(125)


In [2]:
# Datos a utilizar
path_data = "../../../datos/events_anomalydetection.h5"

In [3]:
# Esta celda se corre una vez para pre-procesar los datos
# Una vez que el archivo existe no vuelve a correr
build_features(path_data=path_data, nbatch=11, outname='RnD-1100000', outdir='../../../datos/', chunksize=100000)

A file with that name already exists


In [4]:
# En esta celda preparamos los datos para utilizar los algoritmos
# Separamos los datos en conjuntos de entrenamiento y prueba
df_RnD = pd.read_csv('../../../datos/RnD-1100000.csv')
X, y = separate_data(df_RnD, standarize=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y) # 70% training y 30% test

# Guardamos las columnas de masa
X_train_m = X_train.loc[:,['m_j1', 'm_j2', 'm_jj']].copy()
X_test_m = X_test.loc[:,['m_j1', 'm_j2', 'm_jj']].copy()

# Eliminamos las masas de los conjuntos de entrenamiento y prueba
X_train.drop(['m_j1', 'm_j2', 'm_jj'], axis=1, inplace=True)
X_test.drop(['m_j1', 'm_j2', 'm_jj'], axis=1, inplace=True)

In [5]:
# Listamos los clasificadores a utilizar
# En conjunto con el scaler
classifiers = [(MinMaxScaler(feature_range=(-1,1)), TensorflowClassifier(input_shape = [X_train.shape[1]])),
                (StandardScaler(), RandomForestClassifier(random_state=1)),
                (RobustScaler(), GradientBoostingClassifier(random_state=4)),
                (RobustScaler(), QuadraticDiscriminantAnalysis()), 
                (StandardScaler(), MLPClassifier(random_state=7)),
                (StandardScaler(), KMeans(n_clusters=2, random_state=15))
                ]

In [6]:
# Con esta función entrenamos los modelos
# Solo hace falta correrla una vez
training(X_train, y_train, classifiers, '../../../datos', 'log')


  0%|          | 0/6 [00:00<?, ?it/s]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 14)               56        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 512)               7680      
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 batch_normalization_2 (Batc  (None, 512)              2

100%|██████████| 6/6 [1:29:46<00:00, 897.80s/it]   


Models saved


In [6]:
# En esta celda cargamos los modelos entrenados
models = []

# Cargamos el algoritmo entrenado de tensorflow
tf_model = load_model(os.path.join('../../../datos','tf_model_{}.h5'.format('log')))
models.append(('TensorflowClassifier', tf_model))

# Cargamos los algoritmos entrenados de scikit-learn
with open(os.path.join('../../../datos',"sklearn_models_{}.pckl".format('log')), "rb") as f:
    while True:
        try:
            models.append(pickle.load(f))
        except EOFError:
            break

# Evaluamos
clfs = evaluate(X_test, y_test, models ,train=True)

100%|██████████| 6/6 [01:45<00:00, 17.52s/it]


In [7]:
# En esta celda graficamos las características más importantes para los modelos

# Obtenemos las características más importantes
# models[x][y] : x = 1(random forest), 2(gradient boosting); y = 0(nombre del modelo), 1(modelo entrenado) 
fi_rf = models[1][1].steps[1][1].feature_importances_.tolist()
fi_gb = models[2][1].steps[1][1].feature_importances_.tolist()

# Redondeamos los puntajes
weights = [fi_rf, fi_gb]
weights = [[ round(elem, 3) for elem in weight ] for weight in weights]
# Obtenemos los nombres de los modelos
names_fi = [models[1][0], models[2][0]]
# Obtenemos los nombres de las características
features = X_train.columns.tolist()

# Juntamos los puntajes con las características en tuplas
importance = {}
ii = 0
for weight in weights:
    f_i = list(zip(features, weight))
    importance[names_fi[ii]] = f_i
    ii +=1

# Graficamos en un bucle 
colors=['darkorange', 'green']
lista_images = []
ii = 0
for name, scores in importance.items():

    # Ordenamos de menor a mayor
    scores.sort(key=lambda x: x[1], reverse=False) 

    # Salvamos los nombres y su puntaje separados
    # y revertimos las tuplas para tener de mayor a menor puntaje 
    features = list(zip(*scores))[0]
    score = list(zip(*scores))[1]
    x_pos = np.arange(len(features)) 

    # Graficamos
    fig = plt.figure(facecolor='white')
    plt.barh(x_pos, score,align='center', color = colors[ii])
    plt.yticks(x_pos, features) 
    plt.xlabel('Feature importance')
    plt.title('Feature importance: {}'.format(name))
    filename = '../../figuras/{}-feature-importance.png'.format(name)
    lista_images.append(filename)
    plt.savefig(filename, bbox_inches='tight')
    plt.close('all')
    ii += 1
    
image_grid(rows=1, columns=2, images=lista_images, name='clas-feature-imp', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-feature-imp.png
---
name: clas-feature-imp
width: 80%
---
Importancia de las variables utilizadas en el entrenamiento según RFC (izquierda) y GBC (derecha).
```

Las características más importantes para ambos clasificadores son la variable de subestructura $\tau_{21}$, el $p_T$ de los jets y el número de hadrones de los eventos, y las menos importantes son $\eta$ y $\phi$. Esto indica que las variables con mayor separación entre las distribuciones de señal y fondo, como se puede observar en la {numref}`datospp-vardiff-RnD` del capítulo anterior, son mas relevante en el aprendizaje de los modelos. 
### Distribuciones de eventos clasificados
Con la clasificación realizada por los distintos modelos podemos graficar las distribuciones resultantes de los eventos clasificados como señal o fondo y compararlas con las distribuciones reales de los datos de los eventos clasificados. En la {numref}`clas-variables-dist` observamos las distribuciones de los datos reales y clasificados de las variables más importantes según la {numref}`clas-feature-imp`.

In [8]:
# En esta celda graficamos algunas variables importantes
list_images =[]
variables=['pT_j2', 'tau_21_j1', 'n_hadrons']
for model in clfs:
    for variable in variables:
        # Obtenemos predicciones como pandas Series
        pred = pd.Series(model.pred.flatten(), name='ypred')
        # Obtenemos las etiquetas como pandas Serie
        label = pd.Series(model.label, name='ytest').reset_index(drop=True)
        # Juntamos las masas, las predicciones y las etiquetas
        X_plot = pd.concat([X_test.reset_index(drop=True), X_test_m.reset_index(drop=True), pred, label], axis=1)
        # Graficamos
        fig = plt.figure(facecolor='white')
        pred_test_hist(X_plot, variable, ypred='ypred', ytest='ytest', n_bins=50, log=False)
        plt.title('{}: distribución de {}'.format(model.name, variable))
        # Guardando el path de cada imagen
        filename = os.path.join('../../figuras/','{}-{}.png'.format(model.name,variable))
        list_images.append(filename)
        # Salvamos la imagen como png
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close('all')
        del X_plot, pred, label

image_grid(rows=6, columns=3, images=list_images, name='clas-variables-dist', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-variables-dist.png
---
name: clas-variables-dist
width: 100%
---
Distribución reales y de la clasificación de algunas de las variables más importantes para la clasificación usando el conjunto prueba R&D. Cada fila de imágenes representa un clasificador. De arriba a abajo: Tensorflow Classifier, RFC, GBC, QDA, MLP y KMeans. Por columna, de izquierda a derecha, se encuentras las distribuciones de $p_T$ del jet secundario, $\tau_{21}$ del jet principal y el número de hadrones de los eventos.
```
En general, las distribuciones obtenidas de la clasificación realizada por los modelos es similar los datos reales. La distinción entre señal y fondo es lograda de forma más precisa por los modelos supervisados, porque poseen más información en sobre los eventos en el momento del aprendizaje, puesto que aprenden utilizando la etiqueta de cada evento. 

Particularmente, vemos que el clasificador de Tensorflow está subestimando la cantidad de eventos de señal en todas las variables. Esto se evidencia en los picos de las distribuciones de las variables, donde se observa que las distribuciones de los eventos clasificados como señal poseen menor frecuencia de eventos que la señal real. Al contrario, se observa que RFC, GBC y QDA sobreestiman la cantidad de eventos de señal, puesto que en los picos de señal en cada gráfico se observa que las distribuciones de los eventos clasificados como señal son mas frecuentes que los de la señal real. El clasificador MLP parece clasificar con mayor precisión, ya que la señal clasificada es más cercana a la real. El único modelo no-supervisado, Kmeans, logra separar algunos eventos de señal según los gráficos de $p_T$ y de $\tau_{12}$, pero en el gráfico de número de hadrones vemos que no logra diferenciar entre clases. Esto indica que los eventos de señal predicho están contaminados por eventos de fondo.

Al graficar algunas de las variables con menor importancia según la {numref}`clas-feature-imp`, en la {numref}`clas-variables-noimp-dist`, también se evidencia que el clasificador de TensorFlow está subestimando la cantidad de eventos de señal y que RFC, GBC y QDA sobreestiman la cantidad de eventos de señal. Como se mencionó anteriormente, el modelo MLP clasifica más precisamente, lo que también se observa en estas variables, donde las distribuciones de la clasificación son similares a las de los datos reales. Para KMeans, se observa nuevamente que los eventos predichos como señal está contaminados por eventos de fondo, puesto que en el gráfico de $\eta$ el modelo no distingue entre señal y fondo, y en $\Delta R$ y $E$ los picos de las distribuciones de los eventos predichos como señal muestran menor frecuencia de eventos que las distribuciones de la señal real.

In [9]:
# En esta celda graficamos las variables menos relevantes para la clasificación
list_images =[]
variables=['deltaR_j12', 'E_j1', 'eta_j1']
for model in clfs:
    for variable in variables:
        # Obtenemos predicciones como pandas Series
        pred = pd.Series(model.pred.flatten(), name='ypred')
        # Obtenemos las etiquetas como pandas Serie
        label = pd.Series(model.label, name='ytest').reset_index(drop=True)
        # Juntamos las masas, las predicciones y las etiquetas
        X_plot = pd.concat([X_test.reset_index(drop=True), X_test_m.reset_index(drop=True), pred, label], axis=1)
        # Graficamos
        fig = plt.figure(facecolor='white')
        pred_test_hist(X_plot, variable, ypred='ypred', ytest='ytest', n_bins=50, log=False)
        plt.title('{}: distribución de {}'.format(model.name, variable))
        # Guardando el path de cada imagen
        filename = os.path.join('../../figuras/','{}-{}.png'.format(model.name,variable))
        list_images.append(filename)
        # Salvamos la imagen como png
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close('all')
        del X_plot, pred, label

image_grid(rows=6, columns=3, images=list_images, name='clas-variables-noimp-dist', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-variables-noimp-dist.png
---
name: clas-variables-noimp-dist
width: 100%
---
Distribución de algunas de las variables menos relevantes para la clasificación. Cada fila de imágenes representa un clasificador. De arriba a abajo: Tensorflow Classifier, RFC, GBC, QDA, MLP y KMeans. Por columna, de izquierda a derecha, se encuentras las distribuciones de $\Delta R$, $E$ del jet principal y $\eta$ del jet principal.
```
En la {numref}`clas-masa-dist` podemos ver las distribuciones de la variable de masa invariante de acuerdo a la clasificacion realizada por los modelos. Las variables de masa no fueron utilizadas para entrenamiento y es de las variables más comunmente utilizadas en la búsqueda de nueva física. En esta variable, se observa que los modelos supervisados obtienen resultados más precisos que el modelo no-supervisado.

In [10]:
# En esta celda graficamos la masa invariante predicha
list_mass_images=[]
variables=['m_jj']
for model in clfs:
    for variable in variables:
        # Obtenemos predicciones como pandas Series
        pred = pd.Series(model.pred.flatten(), name='ypred')
        # Obtenemos las etiquetas como pandas Serie
        label = pd.Series(model.label, name='ytest').reset_index(drop=True)
        # Juntamos las masas, las predicciones y las etiquetas
        X_plot = pd.concat([X_test.reset_index(drop=True), X_test_m.reset_index(drop=True), pred, label], axis=1)
        # Graficamos
        pred_test_hist(X_plot, variable, ypred='ypred', ytest='ytest', n_bins=50, log=False)
        plt.title('{}: distribución de {}'.format(model.name, variable))
        # Guardando el path de cada imagen
        filename = os.path.join('../../figuras/','{}-{}.png'.format(model.name,variable))
        list_mass_images.append(filename)
        # Salvamos la imagen como png
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close('all')
        del X_plot, pred, label
    
image_grid(rows=2, columns=3, images=list_mass_images, name='clas-masa-dist', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-masa-dist.png
---
name: clas-masa-dist
width: 100%
---
Distribución la masa invariante y la clasificación para el conjunto R&D. En la fila superior, de izquierda a derecha, las predicciones de: Tensorflow Classifier, RFC, GBC y en la fila inferior: QDA, MLP y KMeans.
```
En los modelos supervisados se vuelve a evidenciar lo discutido anteriormente; el modelo de TensorFlow subestima la cantidad de señal y los demás modelos sobreestiman la cantidad de señal. Kmeans obtiene un pico para la señal clasificada de magnitud correcta, pero con baja frecuencia, y es evidente que está clasificando eventos de fondo como señal, como se puede notar en el gráfico al observar las distribuciones de señal y fondo entre 4000 y 6000 GeV.

(clas-BB1)=
## Conjunto BB1
El conjunto BB1 se clasifica completamente, utilizando los modelos entrenados con el 70% del conjunto R&D. Como se mencionó en capítulos anteriores, este conjunto posee una menor proporción de señal, 0.08% del conjunto es señal, y las partículas de nueva física son más masivas.
### Distribuciones de eventos clasificados
Las distribuciones de las variables más importantes según la {numref}`clas-feature-imp` y la clasificación realizada utilizando el conjunto BB1, se puede observar en la {numref}`clas-variables-dist-BB1`.

In [11]:
# En esta celda preparamos los datos a utilizar por los algoritmos
# Separamos los datos variables y label
df_BB1 = pd.read_csv('../../../datos/BB1-1000000.csv')
X, y = separate_data(df_BB1, standarize=False)

# Guardamos las columnas de masa
X_m = X.loc[:,['m_j1', 'm_j2', 'm_jj']].copy()

# Eliminamos las masas
X.drop(['m_j1', 'm_j2', 'm_jj'], axis=1, inplace=True)

# Realizamos las predicciones
clfs = evaluate(X, y, models ,train=False)

100%|██████████| 6/6 [05:37<00:00, 56.24s/it] 


In [12]:
# En esta celda graficamos algunas variables importantes
list_images =[]
variables=['pT_j2', 'tau_21_j1', 'n_hadrons']
for model in clfs:
    for variable in variables:
        # Obtenemos predicciones como pandas Series
        pred = pd.Series(model.pred.flatten(), name='ypred')
        # Obtenemos las etiquetas como pandas Serie
        label = pd.Series(model.label, name='ytest').reset_index(drop=True)
        # Juntamos las masas, las predicciones y las etiquetas
        X_plot = pd.concat([X.reset_index(drop=True), X_m.reset_index(drop=True), pred, label], axis=1)
        # Graficamos
        fig = plt.figure(facecolor='white')
        pred_test_hist(X_plot, variable, ypred='ypred', ytest='ytest', n_bins=50, log=False)
        plt.title('{}: distribución de {}'.format(model.name, variable))
        # Guardando el path de cada imagen
        filename = os.path.join('../../figuras/','{}-{}.png'.format(model.name,variable))
        list_images.append(filename)
        # Salvamos la imagen como png
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close('all')
        del X_plot, pred, label

image_grid(rows=6, columns=3, images=list_images, name='clas-variables-dist-BB1', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-variables-dist-BB1.png
---
name: clas-variables-dist-BB1
width: 100%
---
Distribución de las variables más importantes y las distribuciones de las clasificaciones para el conjunto BB1. Cada fila de imágenes representa un clasificador. De arriba a abajo: Tensorflow Classifier, RFC, GBC, QDA, MLP y KMeans. Por columna, de izquierda a derecha, se encuentras las distribuciones de $p_T$ del jet secundario, $\tau_{21}$ del jet principal y el número de hadrones de los eventos.
```
El poder de distinción entre señal y fondo de los modelos disminuye al utilizar el conjunto BB1, lo que indica que los modelos no logran generalizar del entrenamiento realizado con el conjunto R&D a otro conjunto de datos con masas distintas para las partículas de BSM. A pesar de haber eliminado la masa para el entrenamiento de los modelos, se utilizaron variables que cambian al variar la masa de las partículas de nueva física del evento. Entre estas variables se encuentra el $p_T$ de los jets, que es mayor para los jets del conjunto BB1 debido a que la partícula de nueva física es más masiva, $\tau_{21}$, debido a que es más difícil obtener subjets de jets provenientes de colisiones con mayor impulso, y $\Delta R$, cuya distribución es más angosta para colisiones más energéticas. Dichas variables se encuentran entre las más importantes para realizar la clasificación de acuerdo a {numref}`clas-feature-imp`, lo que puede afectar la clasificación de datos con distintas masas. La diferencia entre las variables mencionadas anteriormente para la señal en los conjuntos R&D y BB1 se encuentra en la {numref}`clas-senal-RnD-BB1`.

In [15]:
# Cargamos los datos
df_RnD = pd.read_csv('../../../datos/RnD-1100000.csv')
df_BB1 = pd.read_csv('../../../datos/BB1-1000000.csv')
# Obtenemos los datos de señal en cada conjunto de datos
df_RnD_sig = df_RnD.loc[df_RnD.loc[:,'label']==1]
df_BB1_sig = df_BB1.loc[df_BB1.loc[:,'label']==1]
# Graficamos
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(1, 1, 1)    
# Variables a graficar
variables = ['pT_j1', 'tau_21_j1', 'deltaR_j12']
list_images = []
for variable in variables:
    # Creamos los histogramas
    df_RnD_sig[variable].plot.hist(bins=50, facecolor='darkorange', alpha=0.4, label='R&D', density=True)
    df_BB1_sig[variable].plot.hist(bins=50, facecolor='darkturquoise', alpha=0.4, label='BB1', density=True)   
    # Etiquetamos los ejes, colocamos título y leyenda
    plt.xlabel(variable)  
    plt.ylabel('Events density')
    plt.legend(loc='upper right')
    plt.title('Distribution of {}'.format(variable))
    # Guardamos la imagen
    filename = './../../figuras/clas-senal-RnD-BB1-{}.png'.format(variable)
    plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
    # Creamos una lista con las imagenes creadas
    list_images.append(filename)
    plt.close('all')

# Creamos un grid de las imagenes    
image_grid(rows=1, columns=3, images=list_images, name='clas-senal-RnD-BB1', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-senal-RnD-BB1.png
---
name: clas-senal-RnD-BB1
width: 100%
---
Comparación de las distribuciones de señal de las variables $p_T$ del jet principal, $\tau_{21}$ del jet principal y $\Delta R$ de los conjuntos de datos R&D y BB1.
```
Los resultados de la clasificación para las variables más relevantes se pueden visualizar en la {numref}`clas-variables-dist-BB1`. A nivel general, se observa que los modelos no logran realizar correctamente la clasificación de eventos de señal y que dicha clasificación se encuentra contaminada por eventos de fondo. En los gráficos de $p_T$, notamos que los modelos obtienen una distribución corrida hacia la izquierda, es decir, aprendieron a clasificar como señal eventos de jets que poseen menor $p_T$. Igualmente en la variable $\tau_{21}$, si comparamos con la distribución de esta variable en la {numref}`clas-senal-RnD-BB1`, notamos que los modelos están clasificando para una distribución parecida a la del conjunto R&D. 

Las distribuciones de las variables menos relevante para la clasificación se encuentran en la {numref}`clas-variables-noimp-dist-BB1`. Las distribuciones de estas variables refuerza lo analizado anteriormente. En general, los modelos clasifican señal con un $\Delta R$ más ancho y jets menos energéticos, similares a las variables del conjunto R&D.

In [13]:
# En esta celda graficamos las variables menos relevantes para la clasificación
list_images =[]
variables=['deltaR_j12', 'E_j1', 'eta_j1']
for model in clfs:
    for variable in variables:
        # Obtenemos predicciones como pandas Series
        pred = pd.Series(model.pred.flatten(), name='ypred')
        # Obtenemos las etiquetas como pandas Serie
        label = pd.Series(model.label, name='ytest').reset_index(drop=True)
        # Juntamos las masas, las predicciones y las etiquetas
        X_plot = pd.concat([X.reset_index(drop=True), X_m.reset_index(drop=True), pred, label], axis=1)
        # Graficamos
        fig = plt.figure(facecolor='white')
        pred_test_hist(X_plot, variable, ypred='ypred', ytest='ytest', n_bins=50, log=False)
        plt.title('{}: distribución de {}'.format(model.name, variable))
        # Guardando el path de cada imagen
        filename = os.path.join('../../figuras/','{}-{}.png'.format(model.name,variable))
        list_images.append(filename)
        # Salvamos la imagen como png
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close('all')
        del X_plot, pred, label

image_grid(rows=6, columns=3, images=list_images, name='clas-variables-noimp-dist-BB1', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-variables-noimp-dist-BB1.png
---
name: clas-variables-noimp-dist-BB1
width: 100%
---
Distribución real y predicha de algunas de las variables menos relevantes para la clasificación para el conjunto BB1 . Cada fila de imágenes representa un clasificador. De arriba a abajo: Tensorflow Classifier, RFC, GBC, QDA, MLP y KMeans. Por columna, de izquierda a derecha, se encuentras las distribuciones de $\Delta R$, $E$ del jet principal y $\eta$ del jet principal.
```
Por último, en la {numref}`clas-masa-dist-BB1`, observamos las distribuciones de masa invariante obtenidas para este conjunto de datos. Ninguno de los modelos reconstruye la distribución de la señal. Particularmente, notamos que los picos de señal están corridos hacia menores valores de masa, indicando que los modelos aprendieron a clasificar eventos con menor masa invariante, como la señal del conjunto R&D, a pesar de no haber utilizado las variables de masa en el entrenamiento y la clasificación.

En todas las variables el clasificador de TensorFlow y KMeans obtienen las distribuciones de la clasificación menos similares a las reales. Los demás modelos obtuvieron resultados parecidos entre sí, pero en todos es evidente la contaminación de la clasificación de eventos de señal con fondo y viceversa.

In [14]:
# En esta celda graficamos la masa invariante predicha
list_mass_images=[]
variables=['m_jj']
for model in clfs:
    for variable in variables:
        # Obtenemos predicciones como pandas Series
        pred = pd.Series(model.pred.flatten(), name='ypred')
        # Obtenemos las etiquetas como pandas Serie
        label = pd.Series(model.label, name='ytest').reset_index(drop=True)
        # Juntamos las masas, las predicciones y las etiquetas
        X_plot = pd.concat([X.reset_index(drop=True), X_m.reset_index(drop=True), pred, label], axis=1)
        # Graficamos
        pred_test_hist(X_plot, variable, ypred='ypred', ytest='ytest', n_bins=50, log=False)
        plt.title('{}: distribución de {}'.format(model.name, variable))
        # Guardando el path de cada imagen
        filename = os.path.join('../../figuras/','{}-{}.png'.format(model.name,variable))
        list_mass_images.append(filename)
        # Salvamos la imagen como png
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close('all')
        del X_plot, pred, label
    
image_grid(rows=2, columns=3, images=list_mass_images, name='clas-masa-dist-BB1', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-masa-dist-BB1.png
---
name: clas-masa-dist-BB1
width: 100%
---
Distribución la masa invariante y la clasificación del conjunto BB1. En la fila superior, de izquierda a derecha, las predicciones de: Tensorflow Classifier, RFC, GBC y en la fila inferior: QDA, MLP y KMeans.
```
Aunque la comparación de distribuciones proporciona un punto de partida para la comparación de los algoritmos, es necesario el uso de métricas, porque no es evidente cual algoritmo está realizando una mejor clasificación de los conjuntos de datos.